In [115]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import norm
import time

This file gives an alternative implementation of the binomial tree model using ***Binary Trees*** instead of Arrays.

In [116]:
# Definition of the class that models the node and utility functions

class Node:
    
    def __init__(self, up=None, down=None, option=None, stock=None) -> None:
        self.up = up
        self.down = down
        self.option = option
        self.stock = stock
        
def print_tree(node, level=0, prefix="Root: ", is_left=None):
    if node is not None:
        if level == 0:
            print(f"{prefix}Option: {node.option}, Stock: {node.stock}")
        else:
            print(" " * (level - 1) * 6, end="")
            print("|" + ("-- " if is_left else "\\-- ") + f"Option: {node.option}, Stock: {node.stock}")

        if node.up is not None or node.down is not None:
            print_tree(node.up, level + 1, "Up: ", True)
            print_tree(node.down, level + 1, "Down: ", False)

In [117]:
# Import historical data

equity_data = yf.download("RR.L", start="2021-11-27", end="2023-11-27")
equity_data = pd.DataFrame(equity_data['Adj Close'])
log_returns = np.log(equity_data['Adj Close']/equity_data['Adj Close'].shift(1))

# Model parameter

maturity = 0.5 # Time to Maturity in years
S0 = equity_data['Adj Close'].iloc[-1] # last Stock price
K = 200.6 #Strike Price
r = 0.025 #Risk-free rate
sigma = log_returns.std() * np.sqrt(252) #Volatility Rate
payoff = "put"
N = 4

dT = float(maturity) / N   # Time interval
up = np.exp(sigma * np.sqrt(dT))    # Up factor
down = 1.0 / up   # Down factor 

compound_r = np.exp(r * dT)    # risk free compound return
up_prob = (compound_r - down)/ (up - down)  # risk neutral probability of stock going up
down_prob = 1.0 - up_prob           # risk neutral probability of stock going down
print('\n\nRisk-neutral porbability of stock going up is --> ', up_prob.round(4))
print('Risk-neutral porbability of stock going down is --> ', down_prob.round(4))

__exec = 0.0 # a variable used to measure the execution time of this implementation of the binomial trees


[*********************100%%**********************]  1 of 1 completed


Risk-neutral porbability of stock going up is -->  0.4669
Risk-neutral porbability of stock going down is -->  0.5331


In [118]:
__strt = time.time()


# Creating the Tree and Filling with stock prices forecasts

root = Node(stock=S0.round(4))

def forecast_stocks(root, height):
    
    if root is None or height == 0:
        return
    
    root.up = Node(stock=(root.stock * up).round(4))
    root.down = Node(stock=(root.stock * down).round(4))
    
    forecast_stocks(root.up, height - 1)
    forecast_stocks(root.down, height - 1)
    
forecast_stocks(root, N)
print_tree(root)

__end = time.time()
__exec = (__end - __strt)


Root: Option: None, Stock: 241.0
|-- Option: None, Stock: 285.517
      |-- Option: None, Stock: 338.2571
            |-- Option: None, Stock: 400.7392
                  |-- Option: None, Stock: 474.7629
                  |\-- Option: None, Stock: 338.2571
            |\-- Option: None, Stock: 285.517
                  |-- Option: None, Stock: 338.2571
                  |\-- Option: None, Stock: 241.0
      |\-- Option: None, Stock: 241.0
            |-- Option: None, Stock: 285.517
                  |-- Option: None, Stock: 338.2571
                  |\-- Option: None, Stock: 241.0
            |\-- Option: None, Stock: 203.424
                  |-- Option: None, Stock: 241.0
                  |\-- Option: None, Stock: 171.7067
|\-- Option: None, Stock: 203.424
      |-- Option: None, Stock: 241.0
            |-- Option: None, Stock: 285.517
                  |-- Option: None, Stock: 338.2571
                  |\-- Option: None, Stock: 241.0
            |\-- Option: None, Stock: 203.42

In [119]:
__strt = time.time()

# From the forcasted stock, each option value is forcasted

def calc_option(root, payoff):
    
    # Base case where the node is a final node --> stock and option prices at maturity
    if root.up is None or root.down is None:
        if payoff == "call":
            root.option = np.maximum(root.stock - K, 0.0).round(4)  # Call option price at maturity date = payoff of the option at maturity
        elif payoff == "put":
            root.option = np.maximum(K - root.stock, 0.0).round(4)  # Put option price at maturity date = payoff of the option at maturity
        return
    
    # Calculate option price for nodes one step before maturity
    calc_option(root.up, payoff)
    calc_option(root.down, payoff)
    
    # Calculate option price for the root node
    if root.option is None:  # If the root's option price is not calculated yet
        root.option = (np.exp(-r * dT) * ((up_prob * root.up.option) + (down_prob * root.down.option))).round(4)
    
calc_option(root, payoff=payoff)
print_tree(root)

__end = time.time()
__exec += (__end - __strt)

Root: Option: 14.3154, Stock: 241.0
|-- Option: 4.3363, Stock: 285.517
      |-- Option: 0.0, Stock: 338.2571
            |-- Option: 0.0, Stock: 400.7392
                  |-- Option: 0.0, Stock: 474.7629
                  |\-- Option: 0.0, Stock: 338.2571
            |\-- Option: 0.0, Stock: 285.517
                  |-- Option: 0.0, Stock: 338.2571
                  |\-- Option: 0.0, Stock: 241.0
      |\-- Option: 8.1598, Stock: 241.0
            |-- Option: 0.0, Stock: 285.517
                  |-- Option: 0.0, Stock: 338.2571
                  |\-- Option: 0.0, Stock: 241.0
            |\-- Option: 15.3546, Stock: 203.424
                  |-- Option: 0.0, Stock: 241.0
                  |\-- Option: 28.8933, Stock: 171.7067
|\-- Option: 23.1398, Stock: 203.424
      |-- Option: 8.1598, Stock: 241.0
            |-- Option: 0.0, Stock: 285.517
                  |-- Option: 0.0, Stock: 338.2571
                  |\-- Option: 0.0, Stock: 241.0
            |\-- Option: 15.3546, Stock:

In [120]:
print(f'Rolls Royce European Put Option Price with this implementation of Binomial Tree model--> {root.option.round(4)}')

print(f'The total execution time is --> {np.round(__exec, 6)} seconds')

Rolls Royce European Put Option Price with this implementation of Binomial Tree model--> 14.3154
The total execution time is --> 0.005756 seconds
